In [1]:
!pip install statsmodels --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 5.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
from sqlalchemy import create_engine
import numpy as np
import matplotlib
from time import time
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta, date, datetime
import matplotlib.ticker as tkr
from scipy import stats
plt.style.use('fivethirtyeight')
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
matplotlib.rcParams['text.color'] = 'k'
from pylab import rcParams
from statsmodels.tsa.seasonal import STL

%matplotlib inline

In [3]:
from google.colab import files
import io
uploaded = files.upload()

Saving Global DataBase3.csv to Global DataBase3.csv
Saving Global DataBase2 daily.csv to Global DataBase2 daily.csv


In [24]:
#read dataframe from DB
df=pd.read_csv(io.BytesIO(uploaded['Global DataBase3.csv']), index_col='datetime', parse_dates=True, dayfirst=True)

df = df.sort_index()

#Handeling global outliers
GlobalOutliers =  []
for i in range(0):
    GlobalOutliers.append(df.idxmax()['Price - PT [€/MWh]'])
    df.at[df.idxmax()['Price - PT [€/MWh]'], 'Price - PT [€/MWh]'] = np.nan

#Replacing them with the mean of that day
for d in GlobalOutliers:
    sdt = datetime.combine(d, datetime.min.time())
    edt = sdt + timedelta(hours=24)
    df.at[d, 'Price - PT [€/MWh]'] = df[sdt:edt]['Price - PT [€/MWh]'].mean()
    
    
#Handling negative values 
negativeIndex = df.index[df['Price - PT [€/MWh]'] < 0]
for index in (negativeIndex):
    df.at[index, 'Price - PT [€/MWh]']= df.at[index - timedelta(hours=1), 'Price - PT [€/MWh]']

df = df.dropna()

In [25]:
df = df.resample('H').first()

In [26]:
# Set the frequency of the date time index as Monthly start as indicated by the data
df.index.freq = "H"
# Set the value of Alpha and define m (Time Period)
m = 12
alpha = 1/(2*m)

In [27]:
# Split into train and test set
data = df[: '2021-11-01 00:00:00']
test_data = df['2021-11-01 00:00:00': ]

In [28]:
start_time = time()

fitted_model1 = ExponentialSmoothing(data["Price - PT [€/MWh]"],trend='add').fit()

exec_time1 = time()-start_time

In [29]:
test_predictions1 = fitted_model1.forecast(test_data.shape[0])

In [30]:
start_time = time()

fitted_model2 = ExponentialSmoothing(data['Price - PT [€/MWh]'],trend='add',seasonal='add',seasonal_periods=12).fit()

exec_time2 = time()-start_time

In [31]:
test_predictions2 = fitted_model2.forecast(test_data.shape[0])

In [32]:
from sklearn import metrics 
#Calculate MAE, MSE, RMSE, CV DAY
MAE= metrics.mean_absolute_error(test_data['Price - PT [€/MWh]'], test_predictions1)
MSE=metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions1)
CV= (np.sqrt(metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions1))/test_data['Price - PT [€/MWh]'].mean())*100
R2= metrics.r2_score(test_data['Price - PT [€/MWh]'], test_predictions1)


print('*************** Exponential Smoothing Results ***************')
#print('Mean Absolute Error:', MAE)
#print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', np.sqrt(MSE))
print('Coefficient of Variance:',CV)
print('R2:', R2)
print('Execution Time:', '%.5f' %  exec_time1)

*************** Exponential Smoothing Results ***************
Root Mean Squared Error: 844.8015710381179
Coefficient of Variance: 379.4134122105951
R2: -128.18245994617823
Execution Time: 1.43161


In [33]:
from sklearn import metrics 
#Calculate MAE, MSE, RMSE, CV DAY
MAE= metrics.mean_absolute_error(test_data['Price - PT [€/MWh]'], test_predictions2)
MSE=metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions2)
CV= (np.sqrt(metrics.mean_squared_error(test_data['Price - PT [€/MWh]'], test_predictions2))/test_data['Price - PT [€/MWh]'].mean())*100
R2= metrics.r2_score(test_data['Price - PT [€/MWh]'], test_predictions2)


print('*************** Holt Winter Results ***************')
print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)  
print('Root Mean Squared Error:', np.sqrt(MSE))
print('Coefficient of Variance:',CV)
print('R2:', R2)
print('Execution Time:', '%.5f' %  exec_time2)

*************** Holt Winter Results ***************
Mean Absolute Error: 95.4051761247995
Mean Squared Error: 13475.850322636898
Root Mean Squared Error: 116.08553020353956
Coefficient of Variance: 52.135801628159776
R2: -1.4392162422056742
Execution Time: 8.50924
